## Interpreter Settings

In [1]:
import sys
print(sys.version)
print(sys.version_info)
print(sys.api_version)
print(sys.hexversion)

3.6.6 (default, Sep 12 2018, 18:26:19) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]
sys.version_info(major=3, minor=6, micro=6, releaselevel='final', serial=0)
1013
50726640


In [2]:
print(sys.platform)

darwin


In [4]:
print(sys.implementation.name)
print(sys.implementation.version)
print(sys.implementation.cache_tag)

cpython
sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)
cpython-36


CPython Command Line Option Flags  

|Option	|Meaning|
|------|------|
|-B	|do not write .py[co] files on import|
|-b	|issue warnings about converting bytes to string without decoding properly and comparing bytes with strings|  
|-bb	|convert bytes warnings to errors|
|-d	|debug output from parser|
|-E	|ignore PYTHON* environment variables (such as PYTHONPATH)|
|-i	|inspect interactively after running script|
|-O	|optimize generated bytecode slightly|
|-OO	|remove doc-strings in addition to the -O optimizations|
|-s	|do not add user site directory to sys.path|
|-S	|do not run ‘import site’ on initialization|
|-t	|issue warnings about inconsistent tab usage|
|-tt	|issue errors for inconsistent tab usage|
|-v	|verbose|

In [5]:
if sys.flags.bytes_warning:
    print('Warning on bytes/str errors')
if sys.flags.debug:
    print('Debuging')
if sys.flags.inspect:
    print('Will enter interactive mode after running')
if sys.flags.optimize:
    print('Optimizing byte-code')
if sys.flags.dont_write_bytecode:
    print('Not writing byte-code files')
if sys.flags.no_site:
    print('Not importing "site"')
if sys.flags.ignore_environment:
    print('Ignoring environment')
if sys.flags.verbose:
    print('Verbose mode')

```
$ python3 -S -E -b sys_flags.py

Warning on bytes/str errors
Not importing "site"
Ignoring environment
```

In [7]:
print(sys.getfilesystemencoding())
print(sys.getdefaultencoding())

utf-8
utf-8


In [9]:
print(sys.ps1)
print(sys.ps2)

In : 
...: 


In [10]:
class LineCounter:

    def __init__(self):
        self.count = 0

    def __str__(self):
        self.count += 1
        return '({:3d})> '.format(self.count)

```
$ python
Python 3.4.2 (v3.4.2:ab2c023a9432, Oct  5 2014, 20:42:22)
[GCC 4.2.1 (Apple Inc. build 5666) (dot 3)] on darwin
Type "help", "copyright", "credits" or "license" for more
information.
>>> from sys_ps1 import LineCounter
>>> import sys
>>> sys.ps1 = LineCounter()
(  1)>
(  2)>
(  3)>
```

In [11]:
print(sys.executable)
print(sys.prefix)

/usr/local/opt/python3/bin/python3.6
/usr/local/Cellar/python3/3.6.4_1/Frameworks/Python.framework/Versions/3.6


## Runtime Environment

In [12]:
# sys_argv.py
print('Arguments:', sys.argv)

Arguments: ['/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py', '-f', '/Users/hejl/Library/Jupyter/runtime/kernel-016356e5-8850-49d4-a189-6e7ae991a82c.json']


```
$ python3 sys_argv.py

Arguments: ['sys_argv.py']

$ python3 sys_argv.py -v foo blah

Arguments: ['sys_argv.py', '-v', 'foo', 'blah']

$ python3 -u sys_argv.py   // the -u option is understood by the interpreter, and is not passed to the program being run

Arguments: ['sys_argv.py']
```

In [2]:
# sys_stdio.py

print('STATUS: Reading from stdin', file=sys.stderr)

data = sys.stdin.read()

print('STATUS: Writing data to stdout', file=sys.stderr)

sys.stdout.write(data)
sys.stdout.flush()

print('STATUS: Done', file=sys.stderr)

STATUS: Reading from stdin
STATUS: Writing data to stdout
STATUS: Done


```
$ cat sys_stdio.py | python3 -u sys_stdio.py

STATUS: Reading from stdin
STATUS: Writing data to stdout
#!/usr/bin/env python3

#end_pymotw_header
import sys

print('STATUS: Reading from stdin', file=sys.stderr)

data = sys.stdin.read()

print('STATUS: Writing data to stdout', file=sys.stderr)

sys.stdout.write(data)
sys.stdout.flush()

print('STATUS: Done', file=sys.stderr)
STATUS: Done
```

In [3]:
# sys_exit.py
exit_code = int(sys.argv[1])
sys.exit(exit_code)

ValueError: invalid literal for int() with base 10: '-f'

```
$ python3 sys_exit.py 0 ; echo "Exited $?"

Exited 0

$ python3 sys_exit.py 1 ; echo "Exited $?"

Exited 1

```

## Memory Management and Limits

In [4]:
one = []
print(sys.getrefcount(one))
two = one
print(sys.getrefcount(one))
del two 
print(sys.getrefcount(one))

2
3
2


In [5]:
class Myclass:
    pass

objects = [[],(),{},set(),'c','c1234',b'bytes',1,2.3,Myclass, Myclass()]

for obj in objects:
    print(f"{type(obj).__name__} : {sys.getsizeof(obj)}")

list : 64
tuple : 48
dict : 240
set : 224
str : 50
str : 54
bytes : 38
int : 28
float : 24
type : 1056
Myclass : 56


In [8]:
class WithoutAttr:
    pass
class WithAttr:
    def __init__(self,a,b):
        self.a = a
        self.b = b
        return
    
print(sys.getsizeof(WithoutAttr()))
print(sys.getsizeof(WithAttr('a33','b33')))

56
56


In [11]:
class WithoutAttr:
    def __sizeof__(self):
        return object.__sizeof__(self) + sum(sys.getsizeof(v) for v in self.__dict__.values())
class WithAttr:
    def __init__(self,a,b):
        self.a = a
        self.b = b
        return
    def __sizeof__(self):
        return object.__sizeof__(self) + sum(sys.getsizeof(v) for v in self.__dict__.values())
print(sys.getsizeof(WithoutAttr()))
print(sys.getsizeof(WithAttr('a33','b33')))

56
160


In [18]:
print('init recur limit:',sys.getrecursionlimit())
sys.setrecursionlimit(55)
print('modified:',sys.getrecursionlimit())
def gen_recur_error(i):
    print('gen_recur_error:',i)
    gen_recur_error(i+1)
try:
    gen_recur_error(1)
except RecursionError as err:
    print(err)

init recur limit: 100
modified: 55
gen_recur_error: 1
gen_recur_error: 2
gen_recur_error: 3
gen_recur_error: 4
gen_recur_error: 5
gen_recur_error: 6
gen_recur_error: 7
gen_recur_error: 8
gen_recur_error: 9
gen_recur_error: 10
gen_recur_error:maximum recursion depth exceeded in comparison


In [19]:
print(sys.maxsize)
print(sys.maxunicode)

9223372036854775807
1114111


maxsize is the maximum size of a list, dictionary, string, or other data structure dictated by the C interpreter’s size type. maxunicode is the largest integer Unicode point supported by the interpreter as currently configured.

In [20]:
print('Smallest difference (epsilon):', sys.float_info.epsilon)
print()
print('Digits (dig)              :', sys.float_info.dig)
print('Mantissa digits (mant_dig):', sys.float_info.mant_dig)
print()
print('Maximum (max):', sys.float_info.max)
print('Minimum (min):', sys.float_info.min)
print()
print('Radix of exponents (radix):', sys.float_info.radix)
print()
print('Maximum exponent for radix (max_exp):', sys.float_info.max_exp)
print('Minimum exponent for radix (min_exp):', sys.float_info.min_exp)
print()
print('Max. exponent power of 10 (max_10_exp):', sys.float_info.max_10_exp)
print('Min. exponent power of 10 (min_10_exp):', sys.float_info.min_10_exp)
print()
print('Rounding for addition (rounds):', sys.float_info.rounds)

Smallest difference (epsilon): 2.220446049250313e-16

Digits (dig)              : 15
Mantissa digits (mant_dig): 53

Maximum (max): 1.7976931348623157e+308
Minimum (min): 2.2250738585072014e-308

Radix of exponents (radix): 2

Maximum exponent for radix (max_exp): 1024
Minimum exponent for radix (min_exp): -1021

Max. exponent power of 10 (max_10_exp): 308
Min. exponent power of 10 (min_10_exp): -307

Rounding for addition (rounds): 1


In [21]:
print('Number of bits used to hold each digit:',sys.int_info.bits_per_digit)
print('Size in bytes of C type used to hold each digit:',sys.int_info.sizeof_digit)

Number of bits used to hold each digit: 30
Size in bytes of C type used to hold each digit: 4


In [22]:
print(sys.byteorder)

little
